## Задание 1

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf

In [2]:
!hdfs dfsadmin -safemode leave

Safe mode is OFF


In [3]:
conf = (
    SparkConf()
    .set("spark.executor.instances", "2")
    .set("spark.executor.cores", "1")
    .set("spark.executor.memory", "512m")
)

sc = SparkContext(appName="lopatin_spark", master="yarn")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/29 23:37:33 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [7]:
!hdfs dfs -put ml-latest-small .

In [5]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("yarn")
    .appName("lopatin_spark_sql")
    .getOrCreate()
)
spark

In [8]:
ratings_df = (
    spark
    .read
    .format("csv")
    .option("header", "True")
    .load("ml-latest-small/ratings.csv")
)
ratings_df.count()

100836

In [9]:
tags_df = (
    spark
    .read
    .format("csv")
    .option("header", "True")
    .load("ml-latest-small/tags.csv")
)
tags_df.count()

3683

для каждого подсчета было выполнено 3 стейджа, в каждом по одной таски

## Задание 2

In [10]:
import pyspark.sql.functions as f

In [11]:
# количество уникальных фильмов
ratings_df.groupby("movieId").count().count()

9724

In [12]:
# количество уникальных юзеров
ratings_df.groupby("userId").count().count()

610

In [13]:
# количество оценок >= 4.0
ratings_df.filter(f.col("rating") >= 4.0).count()

48580

In [14]:
# топ-100 фильмов с самым высоким рейтингом
(
    ratings_df
    .groupBy('movieId')
    .agg(f.avg(f.col("rating")).alias("avg_rating"))
    .orderBy("avg_rating", ascending=False)
    .show(100)
)

+-------+----------+
|movieId|avg_rating|
+-------+----------+
| 102084|       5.0|
| 126921|       5.0|
|   5723|       5.0|
|    467|       5.0|
|  95149|       5.0|
| 140627|       5.0|
|    633|       5.0|
| 147330|       5.0|
| 103602|       5.0|
|  96608|       5.0|
| 118270|       5.0|
|   1349|       5.0|
| 147410|       5.0|
|    496|       5.0|
| 136355|       5.0|
| 136341|       5.0|
| 173351|       5.0|
|    495|       5.0|
| 131724|       5.0|
|  47736|       5.0|
|   6835|       5.0|
|  69469|       5.0|
| 131098|       5.0|
| 151769|       5.0|
| 172875|       5.0|
|   5745|       5.0|
|  26147|       5.0|
|  73822|       5.0|
|   4495|       5.0|
|    626|       5.0|
|   3939|       5.0|
|   3496|       5.0|
|  44851|       5.0|
| 140265|       5.0|
|  53280|       5.0|
| 107951|       5.0|
| 141718|       5.0|
|  72692|       5.0|
|  27751|       5.0|
| 175397|       5.0|
|    876|       5.0|
|  26073|       5.0|
|  85295|       5.0|
| 113829|       5.0|
| 136834|    

In [17]:
# временная разница между тегированием и оценкой
(
    ratings_df
    .alias("r")
    .join(tags_df.alias("t"), on=["userId", "movieId"], how="inner")
    .select((f.col("t.timestamp") - f.col("r.timestamp")).alias("delta"))
    .select(f.avg(f.col("delta")))
    .show()
)

+--------------------+
|          avg(delta)|
+--------------------+
|2.6243727372266974E7|
+--------------------+



In [16]:
# усредненная по пользователям средняя оценка фильма
(
    ratings_df
    .groupBy("userId")
    .agg(f.avg(f.col("rating")).alias("avg_rating"))
    .select(f.avg(f.col("avg_rating")))
    .show()
)

+------------------+
|   avg(avg_rating)|
+------------------+
|3.6572223377474007|
+------------------+



## Задание 3